In [78]:
import numpy as np
import pandas as pd
import random
from collections import deque

random.seed(261)

In [17]:
def initialise_cycle(size):
    state = np.zeros((size, size), dtype=int)

    for i in range(size):
        if random.random() > 0.5:
            state[i, (i+1)%size] = 1
        else:
            state[(i+1)%size, i] = 1
    
    return state

In [18]:
def get_gen_score_vec(state):
    return np.sum(state, axis=1)

In [19]:
def sample_transition(transition_matrix):
    size = transition_matrix.shape[0]
    idxs = np.array([(i, j) for i in range(size) for j in range(size)], dtype="i,i").astype(object)
    return np.random.choice(idxs, p=np.ravel(transition_matrix))

In [20]:
def bfs(state, source, target):
    visited = [target]
    queue = deque([[target]])

    while queue:
        cur_node = queue.popleft()
        # print(cur_node)
        last_visited = cur_node[-1]
        if last_visited == source:
            return cur_node

        for neighbour in np.nonzero(np.atleast_1d(np.asarray(state[last_visited] > 0)))[0]:
            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(cur_node + [neighbour])
                # print(queue)
    
    return None

In [21]:
def do_transaction(state, solution):
    if len(solution) > 1:
        for (s, t) in zip(solution[:-1], solution[1:]):
            state[s, t] -= 1
            state[t, s] += 1
    return state

In [22]:
transition_matrix = np.array([[0, 1/3, 0], [0, 0, 1/3], [1/3, 0, 0]])

In [23]:
NUM_SIMS = 100
NUM_ITERS = 10000

In [24]:
NUM_NODES = 3

In [25]:
all_sims = []

for i in range(NUM_SIMS):
    if (i+1)%50 == 0:
        print(f"Sim {(i+1):03d}")

    state = initialise_cycle(NUM_NODES)

    all_states = [state.copy()]
    for i in range(NUM_ITERS):
        # if (i+1)%1000 == 0:
        #     print(f"Iter {(i+1):05d}")
        (source, target) = sample_transition(transition_matrix)
        soln = bfs(state, source, target)
        if soln is None:
            all_states.append(state.copy())
        else:
            new_state = do_transaction(state, soln)
            all_states.append(new_state.copy())
            state = new_state
    
    all_sims.append(all_states)

Sim 050
Sim 100


In [49]:
all_freqs = [dict(zip(*np.unique([np.array_str(get_gen_score_vec(s)) for s in states], return_counts = True))) for states in all_sims]

In [72]:
vec_list = ['[1 1 1]',
            '[0 2 1]',
            '[2 1 0]',
            '[1 0 2]',
            '[2 0 1]',
            '[0 1 2]',
            '[1 2 0]']

In [70]:
def get_cumprops(states):
    gen_score_vecs = np.array([np.array_str(get_gen_score_vec(s)) for s in states])
    vals = [(gen_score_vecs == v).astype(int) for v in vec_list]
    cum_vals = [np.divide(np.cumsum(v), (range(1,NUM_ITERS+2))) for v in vals]
    return cum_vals

In [91]:
def make_long(cumprop, i):
    df = pd.DataFrame(dict(zip(vec_list, cumprop)))
    df["iter"] = df.index
    long = pd.melt(df, id_vars="iter")
    long["sim"] = i
    return long

In [73]:
all_freqs = [get_cumprops(states) for states in all_sims]

In [97]:
all_longs = pd.concat([make_long(cumprop, i) for (i, cumprop) in enumerate(all_freqs)])

In [ ]:
all_longs.to_csv("all_longs.csv")